<a href="https://colab.research.google.com/github/karthickrajas/Econometrics/blob/master/Portfolio_management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
# https://blog.quantinsti.com/portfolio-management-strategy-python/
# https://towardsdatascience.com/automating-portfolio-optimization-using-python-9f344b9380b9
# https://www.machinelearningplus.com/machine-learning/portfolio-optimization-python-example/
# https://evgenypogorelov.com/portfolio-rebalancing-python.html

!pip install yfinance

import yfinance as yf
import pandas as pd
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install PyPortfolioOpt

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.cla import CLA
from matplotlib.ticker import FuncFormatter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
data = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = "AAPL MSFT AMZN GOOGL META TSLA NVDA BRK-B JNJ UNH",

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "max",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        # interval = "1m",

        # Whether to ignore timezone when aligning ticker data from 
        # different timezones. Default is True. False may be useful for 
        # minute/hourly data.
        ignore_tz = False,

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all OHLC automatically
        # (optional, default is False)
        auto_adjust = True,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = True,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None
    )

data = data.loc['2008-01-01':,:]
data.index = pd.to_datetime(data.index.date)

[*********************100%***********************]  10 of 10 completed


In [24]:
ls_stocks = "AAPL MSFT AMZN GOOGL META TSLA NVDA BRK-B JNJ UNH".split(' ')
",".join(ls_stocks)

'AAPL,MSFT,AMZN,GOOGL,META,TSLA,NVDA,BRK-B,JNJ,UNH'

In [45]:
cls_price = data.loc[:,data.columns.get_level_values(1)=='Close']
cls_price.columns = cls_price.columns.get_level_values(0)
cls_price 

,UNH,AAPL,META,BRK-B,AMZN,MSFT,GOOGL,TSLA,NVDA,JNJ
2008-01-02,46.735245,5.941450,NaN,92.099998,4.812500,25.928686,17.146896,NaN,7.575687,42.707207
2008-01-03,46.949661,5.944195,NaN,93.000000,4.760500,26.039118,17.150400,NaN,7.516018,42.720169
2008-01-04,46.182709,5.490444,NaN,91.000000,4.439500,25.310276,16.441441,NaN,6.884903,42.661850
2008-01-07,46.908440,5.416954,NaN,88.760002,4.441000,25.479599,16.247498,NaN,6.173462,43.322777
2008-01-08,46.355888,5.222096,NaN,88.300003,4.394000,24.625626,15.807808,NaN,6.304276,43.374603
...,...,...,...,...,...,...,...,...,...,...
2022-10-18,522.210022,143.750000,132.800003,281.670013,116.360001,238.500000,100.769997,220.190002,119.669998,166.009995
2022-10-19,520.320007,143.860001,133.229996,278.339996,115.070000,236.479996,99.629997,222.039993,120.510002,164.690002
2022-10-20,520.880005,143.389999,131.529999,273.529999,115.250000,236.149994,99.970001,207.279999,121.940002,165.110001
2022-10-21,533.729980,147.270004,130.009995,282.510010,119.320000,242.119995,101.129997,214.440002,124.660004,168.710007


In [66]:
mu = expected_returns.mean_historical_return(cls_price)
cov = risk_models.sample_cov(cls_price)
ef = EfficientFrontier(mu, cov)
weights = ef.max_sharpe()
ef.portfolio_performance(verbose=True)

Expected annual return: 29.1%
Annual volatility: 27.9%
Sharpe Ratio: 0.97


(0.2914132173373965, 0.2788612993316328, 0.973291087676606)

In [56]:
log_returns = cls_price.pct_change().apply(lambda x: np.log(1+x))
cov_matrix = log_returns.cov()
corr_matrix = log_returns.corr()


In [59]:
cls_price.columns

Index(['UNH', 'AAPL', 'META', 'BRK-B', 'AMZN', 'MSFT', 'GOOGL', 'TSLA', 'NVDA',
       'JNJ'],
      dtype='object')

In [71]:
w = {i:weights[i] for i in weights}
port_var = cov_matrix.mul(w, axis=0).mul(w, axis=1).sum().sum()
(port_var*250)**0.5

0.27768605558488035

In [116]:
rebalancing_dates = data.resample('M').apply(lambda x: x[-1]).index
rebalancing_dates[0]

Timestamp('2008-01-31 00:00:00', freq='M')

In [117]:
data.loc['2008-01-31'].pct_change()
np.log(data.loc['2008-01-31']) - np.log(data.loc['2008-01-31'].shift(1))

TSLA   Open            NaN
       High            NaN
       Low             NaN
       Close           NaN
       Volume          NaN
AMZN   Open            NaN
       High       0.135000
       Low       -0.136016
       Close      0.121070
       Volume    19.212663
NVDA   Open     -18.837281
       High       0.016503
       Low       -0.049929
       Close      0.031395
       Volume    16.160382
UNH    Open     -14.170180
       High       0.020754
       Low       -0.035846
       Close      0.030354
       Volume    12.203259
JNJ    Open     -12.248245
       High       0.024432
       Low       -0.026698
       Close      0.022746
       Volume    13.161574
MSFT   Open     -13.716350
       High       0.025678
       Low       -0.031650
       Close      0.027365
       Volume    15.278407
GOOGL  Open     -15.854600
       High       0.061152
       Low       -0.069947
       Close      0.054647
       Volume    17.555116
BRK-B  Open     -15.657403
       High       0.000000
 

In [120]:
def get_closing_values(data :pd.DataFrame, 
                       date: str
                       ) -> pd.DataFrame:
                       limited_data = data.loc[ date,:]
                       limited_data = limited_data.loc[limited_data.index.get_level_values(1)=='Close',:]
                       limited_data.index = [i[0] for i in limited_data.index]
                       return pd.DataFrame(limited_data.loc[ls_stocks])

In [72]:
ls_stocks
port_wt = {i: round(1/(1+len(ls_stocks)),3) for i in ls_stocks}
port_wt = pd.DataFrame({i:[j] for (i,j) in port_wt.items()}).T 

In [73]:
port_wt.T 

,AAPL,MSFT,AMZN,GOOGL,META,TSLA,NVDA,BRK-B,JNJ,UNH
0,0.091,0.091,0.091,0.091,0.091,0.091,0.091,0.091,0.091,0.091


In [123]:
check = get_closing_values(data, '2016-01-21')
check.T.reset_index(drop=True)

,AAPL,MSFT,AMZN,GOOGL,META,TSLA,NVDA,BRK-B,JNJ,UNH
0,22.076778,45.385151,28.750999,36.3335,94.160004,13.331333,6.789661,125.620003,80.192268,102.458839


In [124]:
tot_value = port_wt.T   * check.T.reset_index(drop=True)

In [125]:
tot_value

,AAPL,MSFT,AMZN,GOOGL,META,TSLA,NVDA,BRK-B,JNJ,UNH
0,2.008987,4.130049,2.616341,3.306348,8.56856,1.213151,0.617859,11.43142,7.297496,9.323754
